# Maximum Likelihood Estimation For Quadratic Discriminant Analysis

Assume that there is a discrete generative model. A classification is to be made using this model. The probability density function of the output to be in the class $c$ is given as follows:
$$
p\left (y=c|\textbf{x}, \boldsymbol{\theta}\right )=\frac{p\left (\textbf{x}|y=c, \boldsymbol{\theta} \right )p\left (y=c|\boldsymbol{\theta} \right )}{p\left ( \textbf{x}|\boldsymbol{\theta}\right )}
$$
If $p\left (\textbf{x}|y=c, \boldsymbol{\theta} \right )$ is given by a multivariate Gaussian distribution, then this classification is called the quadratic discriminant classification. The maximum likelihood estimator for the quadratic discriminant classifier is to be derived. Hence, the likelihood of the data should be written first. The likelihood of a single data sample $\left (\textbf{x}_{i}, y_{i} \right )$ is as follows:
$$
p\left (\textbf{x}_{i}, y_{i}|\boldsymbol{\theta}\right )=p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta} \right ) p\left (y_{i}|\boldsymbol{\theta} \right )=\prod_{c=1}^{C} p\left (y_{i}|\boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}\prod_{c=1}^{C}p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}
$$
Assuming that the samples are independent and there are $N$ training samples, the likelihood of the data can now be written as follows:
$$
p\left (D|\boldsymbol{\theta} \right )=\prod_{i=1}^{N} p\left (\textbf{x}_{i}, y_{i}|\boldsymbol{\theta} \right )
$$
For the sake of avoiding any numerical underflow and computational simplicity, the logarithm of the above expression is taken to obtain the log-likelihood:
$$
\log p\left (D|\boldsymbol{\theta} \right )=\sum_{i=1}^{N} \log p\left (\textbf{x}_{i}, y_{i}|\boldsymbol{\theta} \right )=\sum_{i=1}^{N} \log \left [\prod_{c=1}^{C} p\left (y_{i}|\boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}\prod_{c=1}^{C}p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}\right ]=
$$
$$
\sum_{i=1}^{N} \left [\sum_{c=1}^{C} \log \left [ p\left (y_{i}|\boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}\right ]+\sum_{c=1}^{C}\log \left [p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta}_{c} \right )^{I\left (y_{i}=c \right )}\right ]\right ]=\sum_{i=1}^{N} \left [\sum_{c=1}^{C} I\left (y_{i}=c \right )\log p\left (y_{i}|\boldsymbol{\theta}_{c} \right ) +\sum_{c=1}^{C}I\left (y_{i}=c \right )\log p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta}_{c} \right )\right ]\Rightarrow
$$
$$
\log p\left (D|\boldsymbol{\theta} \right )=\sum_{c=1}^{C} N_{c}\log p\left (y_{i}=c|\boldsymbol{\theta}_{c} \right ) +\sum_{c=1}^{C} \sum_{i:y_{i}=c}\log p\left (\textbf{x}_{i}|y_{i}, \boldsymbol{\theta}_{c} \right )
$$
The first term is maximized if the maximum likelihood estimator (MLE) for the class occurrences which is
$$
\pi_{c_{MLE}}=\frac{N_{c}}{N}
$$
is plugged in place of $p\left (y_{i}=c|\boldsymbol{\theta}_{c} \right )$. $N_{c}$ is the number of occurrences of the class $c$. The second term is maximized if the maximum likelihood estimators for the means and covariances of the multivariate Gaussian distributions for the probability densities of the input features conditioned on the class label are used. The MLE's for the mean and covariances are as follows:
$$
\boldsymbol{\mu}_{c_{MLE}}=\frac{1}{N_{c}}\sum_{i:y_{i}=c} x_{i}
$$
$$
\boldsymbol{\Sigma}_{c_{MLE}} = \frac{1}{N_{c}} \sum_{i:y_{i}=c} \left (\textbf{x}_{i}-\boldsymbol{\mu}_{c_{MLE}} \right )\left (\textbf{x}_{i}-\boldsymbol{\mu}_{c_{MLE}} \right )^{T}
$$
The maximum likelihood estimation measure for $p\left (y=c|\textbf{x}, \boldsymbol{\theta}\right )$ is calculated without computing $p\left ( \textbf{x}|\boldsymbol{\theta}\right )$ since it is the same for all of the classes.

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

In [2]:
class CategoryGaussianData:
    def __init__(self, dist_mean, nonsingular_matrix, dist_cov_eigvalues, num_of_samples):
        # 1-d numpy array for the mean of the distribution
        self.dist_mean = dist_mean
        # a nonsingular matrix to create the eigenvector matrix of the covariance matrix for the distribution
        self.nonsingular_matrix = nonsingular_matrix
        # 1-d array with eigenvalues to be used for the generation of the covariance matrix for the distribution
        self.dist_cov_eigvalues = dist_cov_eigvalues
        # number of samples from the distribution
        self.num_of_samples = num_of_samples
        
    def compute_cov_matrix(self):
        nonsingular_matrix = self.nonsingular_matrix
        q, r = np.linalg.qr(nonsingular_matrix)
        # normalize the eigenvector matrix of the covariance matrix for the distribution
        for i in range(0, q.ndim):
            q[i, :] = q[i, :] / np.linalg.norm(q[i, :])
        eigenvalue_matrix = np.diag(self.dist_cov_eigvalues)
        dist_cov_matrix = np.matmul(np.matmul(q, eigenvalue_matrix), q.T)
        
        return dist_cov_matrix
    
    
    def sample_data(self):
        dist_cov_matrix = self.compute_cov_matrix() 
        
        return np.random.default_rng().multivariate_normal(mean=self.dist_mean, cov=dist_cov_matrix, size=self.num_of_samples)


class MLEComputation:
    def __init__(self, train_input, pi_class_mle):
        # input data for training
        self.train_input = train_input
        # mle for class density
        self.pi_class_mle = pi_class_mle
    
    def gaussian_mle_estimates(self):
        mean_mle = np.mean(self.train_input, axis=0)
        num_of_samples = self.train_input.shape[0]
        num_of_features = self.train_input.shape[1]
        cov_mle = np.zeros((num_of_features, num_of_features))
        for i in range(0, num_of_samples):
            centered_sample = self.train_input[i, :]-mean_mle
            cov_mle = cov_mle + np.outer(centered_sample, centered_sample)
        cov_mle = cov_mle/num_of_samples
        
        return mean_mle, cov_mle
    
    def compute_class_density_measure(self, x):
        mean_mle, cov_mle = self.gaussian_mle_estimates()
        return (multivariate_normal.pdf(x, mean=mean_mle, cov=cov_mle))*self.pi_class_mle

In [3]:
mean_0 = np.array([1, 2, 3, 4])
print('the mean of the category 0 is:')
print(mean_0)
non_singular_0 = np.array([[1, 2, 0, 0], [2, -1, 0, 0], [0, 4, 3, 0], [0, 0, 2.5, 5]])
print('nonsingular matrix for generating the eigenvector matrix of the covariance of the category 0 is:')
print(non_singular_0)
Lambda_0 = np.array([1, 2.4, 3, 3.8])
print('eigenvalues of the covariance matrix of the distribution for the category 0:')
print(Lambda_0)
num_of_samples_0 = 1000
category_0 = CategoryGaussianData(mean_0, non_singular_0, Lambda_0, num_of_samples_0)
cov_0 = category_0.compute_cov_matrix() 
print('covariance matrix of the category 0')
print(cov_0)
print('verification for the eigenvalues of the covariance matrix of the category 0')
print(np.linalg.eigvalsh(cov_0))

the mean of the category 0 is:
[1 2 3 4]
nonsingular matrix for generating the eigenvector matrix of the covariance of the category 0 is:
[[ 1.   2.   0.   0. ]
 [ 2.  -1.   0.   0. ]
 [ 0.   4.   3.   0. ]
 [ 0.   0.   2.5  5. ]]
eigenvalues of the covariance matrix of the distribution for the category 0:
[1.  2.4 3.  3.8]
covariance matrix of the category 0
[[ 2.84883485 -0.92441743 -0.45552178  0.27234043]
 [-0.92441743  1.46220871  0.22776089 -0.13617021]
 [-0.45552178  0.22776089  2.68470111 -0.17021277]
 [ 0.27234043 -0.13617021 -0.17021277  3.20425532]]
verification for the eigenvalues of the covariance matrix of the category 0
[1.  2.4 3.  3.8]


In [4]:
mean_1 = np.array([5, 6, 7, 8])
print('the mean of the category 1 is:')
print(mean_1)
non_singular_1 = np.array([[1, 2, 0, 0], [2, -1, 0, 0], [0, 4, 3, 0], [0, 0, 2.5, 5]])
print('nonsingular matrix for generating the eigenvector matrix of the covariance of the category 1 is:')
print(non_singular_1)
Lambda_1 = np.array([1.5, 2.8, 3.3, 4.6])
print('eigenvalues of the covariance matrix of the distribution for the category 1:')
print(Lambda_1)
num_of_samples_1 = 1000
category_1 = CategoryGaussianData(mean_1, non_singular_1, Lambda_1, num_of_samples_1)
cov_1 = category_1.compute_cov_matrix() 
print('covariance matrix of the category 1:')
print(cov_1)
print('verification for the eigenvalues of the covariance matrix of the category 1:')
print(np.linalg.eigvalsh(cov_1))

the mean of the category 1 is:
[5 6 7 8]
nonsingular matrix for generating the eigenvector matrix of the covariance of the category 1 is:
[[ 1.   2.   0.   0. ]
 [ 2.  -1.   0.   0. ]
 [ 0.   4.   3.   0. ]
 [ 0.   0.   2.5  5. ]]
eigenvalues of the covariance matrix of the distribution for the category 1:
[1.5 2.8 3.3 4.6]
covariance matrix of the category 1:
[[ 3.43483283 -0.96741641 -0.55927052  0.44255319]
 [-0.96741641  1.98370821  0.27963526 -0.2212766 ]
 [-0.55927052  0.27963526  3.14954407 -0.27659574]
 [ 0.44255319 -0.2212766  -0.27659574  3.63191489]]
verification for the eigenvalues of the covariance matrix of the category 1:
[1.5 2.8 3.3 4.6]


In [5]:
train_data_0 = category_0.sample_data()
pi_c_0 = 1000 / 2000
mle_0 = MLEComputation(train_data_0, pi_c_0)
mean_mle, cov_mle = mle_0.gaussian_mle_estimates()
print('mle estimate of the mean for the distribution of category 0:')
print(mean_mle)
print('mle estimate of the covariance for the distribution of category 0:')
print(cov_mle)
print('eigenvalues of the mle of the covariance matrix for the distribution of category 0:')
print(np.linalg.eigvalsh(cov_mle))

mle estimate of the mean for the distribution of category 0:
[0.97241857 1.97843254 2.96578103 4.11295877]
mle estimate of the covariance for the distribution of category 0:
[[ 3.13059469 -0.9204429  -0.50956809  0.03607638]
 [-0.9204429   1.31317454  0.25156013 -0.00793287]
 [-0.50956809  0.25156013  2.76000462 -0.19320975]
 [ 0.03607638 -0.00793287 -0.19320975  3.3670161 ]]
eigenvalues of the mle of the covariance matrix for the distribution of category 0:
[0.9276648  2.43160125 3.3599045  3.8516194 ]


In [6]:
train_data_1 = category_1.sample_data()
pi_c_1 = 1000 / 2000
mle_1 = MLEComputation(train_data_1, pi_c_1)
mean_mle, cov_mle = mle_1.gaussian_mle_estimates()
print('mle estimate of the mean for the distribution of category 1:')
print(mean_mle)
print('mle estimate of the covariance for the distribution of category 1:')
print(cov_mle)
print('eigenvalues of the mle of the covariance matrix for the distribution of category 1:')
print(np.linalg.eigvalsh(cov_mle))

mle estimate of the mean for the distribution of category 1:
[4.94527014 6.04143632 7.02131479 8.03937993]
mle estimate of the covariance for the distribution of category 1:
[[ 3.54744134 -1.04517846 -0.51762962  0.38783089]
 [-1.04517846  2.02292882  0.30279074 -0.35542864]
 [-0.51762962  0.30279074  3.13704031 -0.26085146]
 [ 0.38783089 -0.35542864 -0.26085146  3.56680111]]
eigenvalues of the mle of the covariance matrix for the distribution of category 1:
[1.48113754 2.84650426 3.27940251 4.66716727]


In [7]:
test_input = np.array([3, 4, 5, 6])
print('mle of the class density measure for the class 0:', mle_0.compute_class_density_measure(test_input))
print('mle of the class density measure for the class 1:', mle_1.compute_class_density_measure(test_input))

mle of the class density measure for the class 0: 1.0842533040971799e-05
mle of the class density measure for the class 1: 2.870042120886909e-05


From the above two results, it is observed that the mle's for the probability density measures of the two classes are close to each other for the test input $\left [3, 4, 5, 6 \right ]$. This test input is the half-way between the means of the distributions for the two classes. 

In [8]:
mle_0 = MLEComputation(train_data_0, pi_c_0)
mle_1 = MLEComputation(train_data_1, pi_c_1)
test_input = np.array([1, 2, 3, 4])
print('mle of the class density measure for the class 0:', mle_0.compute_class_density_measure(test_input))
print('mle of the class density measure for the class 1:', mle_1.compute_class_density_measure(test_input))

mle of the class density measure for the class 0: 0.0023380151720176056
mle of the class density measure for the class 1: 1.9083549472588822e-10


For the test input being equal to the mean of the distribution for the class 0, the above results are in compliance with the expectation that category 0 should be chosen by the mle.

In [9]:
mle_0 = MLEComputation(train_data_0, pi_c_0)
mle_1 = MLEComputation(train_data_1, pi_c_1)
test_input = np.array([5, 6, 7, 8])
print('mle of the class density measure for the class 0:', mle_0.compute_class_density_measure(test_input))
print('mle of the class density measure for the class 1:', mle_1.compute_class_density_measure(test_input))

mle of the class density measure for the class 0: 1.1931697725230098e-12
mle of the class density measure for the class 1: 0.0015750917957625316


For the test input being equal to the mean of the distribution for the class 1, the above results are in compliance with the expectation that category 1 should be chosen by the mle.

# References
Machine Learning A Probabilistic Perspective, Kevin P. Murphy